In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install numpy 
!pip install pandas
!pip install sklearn
!pip install torch 
!pip install sklearn
!pip install seaborn
!pip install matplotlib
!pip install json
!pip install fast_ml

     |████████████████████████████████| 4.0 MB 13.8 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 6.6 MB 74.4 MB/s 
     |████████████████████████████████| 880 kB 55.1 MB/s 
     |████████████████████████████████| 596 kB 70.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=ef593bec0dbab1566d20dc07d0d43ee227dcb315003586c6e414304f97a6654e
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
     |████████████████████████████████| 42 kB 977 kB/s 


In [ ]:
import numpy as np #Für Matrizen
import pandas as pd #Für Dataframes
import re
from typing import List, Any, Set, Dict, Tuple, Optional
from ast import literal_eval
import json # load json module
from sklearn.model_selection import train_test_split
import torch
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset
from transformers import BertTokenizer, AutoTokenizer
from transformers import BertForSequenceClassification
from fast_ml.model_development import train_valid_test_split

In [ ]:
dict_name = "l_dict_m3"
json_dir = f'/content/drive/MyDrive/{dict_name}.json'

In [ ]:
with open(json_dir) as jsonFile:
    labels_Input = json.load(jsonFile)
    jsonFile.close()
print(max(labels_Input), min(labels_Input), len(labels_Input))

9724000 -1 369


In [ ]:
labels_Output = {}
for k, v in labels_Input.items():
    labels_Output[v["index"]] = {
            "name": v["name"],
            "code": k,
            "index": v["index"]
            } 
print(len(labels_Output))

369


**Trainingsdaten vorbereiten**

In [ ]:
excel_path = '/content/drive/MyDrive/train_ready_data_m3_v1.xlsx'
sheet_name = "train_ready_dataset_m3_v1"

In [ ]:
df = pd.read_excel(excel_path, sheet_name, index_col=None, dtype={'Text': str, 'Label': np.int64})
df = df.astype({'Label': str})
print(df)
print(len(df))

                                                   Text      Label  \
0     Withdrawal arrhythmia  Injury of intercostal a...         -1   
1     Typhlolithiasis  Injury of ascending right col...         -1   
2     Severe manic bipolar I disorder without psycho...         -1   
3     Poisoning by sawfly larvae  Hemorrhagic bronch...         -1   
4     Miscarriage with laceration of cervix  Chronic...         -1   
...                                                 ...        ...   
2468                Stress echocardiography (procedure)  816996009   
2469                Stress echocardiography (procedure)  816996009   
2470                            Stress echocardiography  816996009   
2471  Regular participation in cardiological special...  816996009   
2472  Performing or assisting with stress echocardio...  816996009   

      label_code data_type  
0              0     train  
1              0     train  
2              0     train  
3              0     train  
4             

In [ ]:
df['Label'].value_counts()

363704007    59
387713003    19
405815000    17
441619002    17
-1           13
             ..
71861002      1
424226004     1
65818007      1
61160002      1
785804000     1
Name: Label, Length: 410, dtype: int64

In [ ]:
df['label_code'].value_counts()

0      217
317     19
348     17
160     11
133     11
      ... 
107      4
338      4
311      4
286      4
339      4
Name: label_code, Length: 369, dtype: int64

**Training vorbereiten**

In [ ]:
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.2')

print("Encode Train Data.......")
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)
print("Encode Validation Data.......")
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)
print("Encode Test Data.......")
encoded_data_test = tokenizer.batch_encode_plus(
    df[df.data_type=='test'].Text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label_code.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label_code.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df[df.data_type=='test'].label_code.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Encode Train Data.......
Encode Validation Data.......
Encode Test Data.......


In [ ]:
model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-base-cased-v1.2',
                                                      num_labels=len(labels_Input),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [ ]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import Adam

optimizer = Adam(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs =  100

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
import random
import torch
import numpy as np

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        #print(loss)
        logits = outputs[1]
        #print(logits)
        loss_val_total += loss.item()
        #print(loss_val_total)

        logits = logits.detach().cpu().numpy()
        #print(logits)
        label_ids = inputs['labels'].cpu().numpy()
        #print(label_ids)

        #print(label_ids)
        predictions.append(logits)
        true_vals.append(label_ids)


    loss_val_avg = loss_val_total/len(dataloader_val) 
    #print(predictions)
    #print(true_vals)
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    #print(len(predictions))
    #print(len(true_vals))

            
    return loss_val_avg, predictions, true_vals

In [ ]:
import random
import torch
import numpy as np

def test(dataloader_test):

    model.eval()
    
    predictions, true_vals = [], []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)     

        logits = outputs[1]

        logits = logits.detach().cpu().numpy()

        label_ids = inputs['labels'].cpu().numpy()

        predictions.append(logits)
        true_vals.append(label_ids)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return predictions, true_vals

**Training durchführen**

In [ ]:
training_losses = []
valid_losses = []
fOnes = []
# set initial loss to infinite
best_valid_loss = float('inf')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    training_losses.append(loss_train_avg)
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(true_vals, predictions)
    tqdm.write(f'Validation loss: {val_loss}')
    valid_losses.append(val_loss)
    
    if val_loss < best_valid_loss:
        best_valid_loss = val_loss
        torch.save(model.state_dict(), '/content/drive/MyDrive/saved_weights_M3_v1.model')  
        
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    fOnes.append(val_f1)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/116 [00:00<?, ?it/s]


Epoch 1
Training loss: 5.861940552448404
Validation loss: 5.721260213851929
F1 Score (Weighted): 0.009756727346502148


Epoch 2:   0%|          | 0/116 [00:00<?, ?it/s]


Epoch 2
Training loss: 5.611858388473248
Validation loss: 5.577335548400879
F1 Score (Weighted): 0.009931474701961889


...


Epoch 98
Training loss: 1.0713172750226383
Validation loss: 1.8661998838186264
F1 Score (Weighted): 0.727615021676265


Epoch 99:   0%|          | 0/116 [00:00<?, ?it/s]


Epoch 99
Training loss: 1.0645773919491932
Validation loss: 1.8656093806028367
F1 Score (Weighted): 0.727615021676265


Epoch 100:   0%|          | 0/116 [00:00<?, ?it/s]


Epoch 100
Training loss: 1.054314559903638
Validation loss: 1.8650015830993651
F1 Score (Weighted): 0.727615021676265


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/saved_weights_M3_v1.model')  

In [ ]:
print(training_losses)
print(valid_losses)
print(fOnes)

In [ ]:
for i in training_losses:
  print(i)

5.861940552448404
5.611858388473248
5.440404813865135
5.301859530909308
5.156311955945245
5.006131233840153
4.878807717356189
4.742190340469623
4.621410295881074
4.480620725401517
4.376016298244739
4.2477364457886795
4.146372860875623
4.050542243595781
3.9296526559467972
3.8344829719642113
3.733252391732972
3.6417682828574347
3.5454863679820092
3.4674635303431542
3.365836597722152
3.288878580619549
3.2036583649701087
3.1363943075311593
3.0402613384970305
2.977038586961812
2.9119150391940414
2.828006156559648
2.7691946296856322
2.698246060774244
2.6256786584854126
2.5639656329977103
2.5117404296480377
2.4445135408434373
2.376497518399666
2.328277509787987
2.280292936440172
2.2162138388074677
2.1743610419076065
2.1154175996780396
2.0730041738214164
2.0260551027182876
1.9956624703160648
1.9499256353953789
1.9155405533724819
1.8587837023981686
1.8164189149593484
1.8047103614642703
1.7581916841967353
1.7338929834036991
1.690638698380569
1.6570945762354752
1.6286376313916568
1.59652052048979

In [ ]:
for i in valid_losses:
  print(i)

5.721260213851929
5.577335548400879
5.468393421173095
5.3687718391418455
5.228316831588745
5.103595900535583
4.977749538421631
4.873497474193573
4.776812541484833
4.664246368408203
4.583797812461853
4.466072082519531
4.3844913125038145
4.292977082729339
4.195368468761444
4.122726607322693
4.038244521617889
3.971001756191254
3.8824102878570557
3.8080387473106385
3.75329076051712
3.6704012513160706
3.5920515418052674
3.552751362323761
3.471322512626648
3.415224885940552
3.364971375465393
3.299064338207245
3.2408979654312136
3.1858105182647707
3.1462487459182737
3.0823858618736266
3.0325701355934145
2.980445849895477
2.927946627140045
2.878143274784088
2.8433343052864073
2.813960838317871
2.7450610637664794
2.7242686986923217
2.6788455843925476
2.6434247136116027
2.621426022052765
2.5699193596839907
2.5549045324325563
2.5156278371810914
2.470792704820633
2.4454588651657105
2.432339835166931
2.398906630277634
2.3832991302013395
2.342376559972763
2.3122859060764314
2.288538706302643
2.27178

In [ ]:
for i in fOnes:
  print(i)

0.009756727346502148
0.009931474701961889
0.010364623131330943
0.013894464516704579
0.024156931090993646
0.0715492324465731
0.11843888277252614
0.1513704927429821
0.18972097434361582
0.23990416292303085
0.2808213536987122
0.31855595487670957
0.3771815186909527
0.3995882000598982
0.41633684275193705
0.47433991247137924
0.49568937351956216
0.513298755751586
0.5437202210787117
0.5601536211969065
0.5295847059998003
0.5772386942198263
0.5860219445125106
0.5849805330937407
0.6061695118298892
0.5956126697290663
0.6107966457023061
0.6147648996705601
0.6014057352500748
0.6234288011749888
0.6216831386642707
0.6327297440504988
0.6228073812979473
0.623418313866427
0.6154518568433663
0.6378933232858979
0.6370170709793351
0.6473272855500488
0.6543445814246928
0.6537755487801884
0.6559169084687179
0.6555245912457297
0.6577707727371942
0.6671115555169116
0.6612897904071553
0.6690591568104715
0.6735269713617996
0.6851679564095876
0.6690915274541753
0.6823000898472598
0.6677018740272684
0.67615545098755

In [ ]:
import matplotlib.pyplot as plt
# line 1 points
x1 = range(len(training_losses))
y1 = training_losses
# plotting the line 1 points 
plt.plot(x1, y1, label = "line 1")
# line 2 points
x2 = range(len(valid_losses))
y2 = valid_losses
# plotting the line 2 points 
plt.plot(x2, y2, label = "line 2")
plt.xlabel('Epochen')
# Set the y axis label of the current axis.
plt.ylabel('Training')
# Set a title of the current axes.
plt.title('Losses')
# show a legend on the plot
plt.legend()
# Display a figure.
plt.show()

In [ ]:
# line 1 points
x1 = range(len(fOnes))
y1 = fOnes
# plotting the line 1 points 
plt.plot(x1, y1, label = "line 1")
plt.xlabel('Epochen')
# Set the y axis label of the current axis.
plt.ylabel('F1')
# Set a title of the current axes.
plt.title('F1 Validation')
# show a legend on the plot
plt.legend()
# Display a figure.
plt.show()

**Testen**

In [ ]:
bert_base_model = "dmis-lab/biobert-base-cased-v1.2"
training_state = '/content/drive/MyDrive/saved_weights_M3_v1.model'

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

model = BertForSequenceClassification.from_pretrained(bert_base_model,
                                                      num_labels=len(labels_Input),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
print("BFSC as Model loaded....")

model.load_state_dict(torch.load(training_state, map_location=device))
print("state_dict loaded.....")

#tokenizer = BertTokenizer.from_pretrained(bert_base_model)

#print("Tokenizer loaded.....")

In [ ]:
predictions, true_vals = test(dataloader_test)

In [ ]:
val_f1 = f1_score_func(true_vals, predictions)
tqdm.write(f'F1 Score (Weighted): {val_f1}')

F1 Score (Weighted): 0.6294512741855492


In [ ]:
preds = np.argmax(predictions, axis = 1)
print(classification_report(true_vals, preds))

              precision    recall  f1-score   support

           0       0.44      0.78      0.56        32
           1       1.00      0.50      0.67         2
           3       1.00      1.00      1.00         2
           4       1.00      0.50      0.67         2
           5       1.00      1.00      1.00         1
           6       0.00      0.00      0.00         1
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         1
           9       0.67      1.00      0.80         2
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         2
          18       1.00      1.00      1.00         1
          20       0.00      0.00      0.00         2
          21       0.50      0.50      0.50         2
          22       0.00      0.00      0.00         1
          24       0.50      1.00      0.67         1
          26       0.00      0.00      0.00         1
          27       0.50    

In [ ]:
sentence = "Comprehensive care of lower urinary tract infections"
model.eval()

input = tokenizer.encode_plus(sentence,
                              add_special_tokens=True,
                              return_attention_mask=True,
                              pad_to_max_length=True,
                              max_length=512,
                              return_tensors='pt')
output = model(**input)
logits = output.logits
soft = torch.nn.Softmax(dim=0)
soft_tensor = soft(logits[0])
probs = soft_tensor.detach().numpy()
#print(probs)
probabilities = {}
for k, v in labels_Input.items():
    probabilities[k] = probs[v["index"]]
print(probabilities)
print("-----------------------------------------------------------------------------------------")
